## Neural networks models: practicals

In this notebook you will practice simple neural networks models for classification. 
We will be using the `breast cancer` dataset:

- binary classification problem: breast cancer diagnosis, `0`: `malignant`, `1`: `benign`
- EDA: look at the data
- split between the training and the test sets
- number of hidden layers
- number of nodes within layers
- type of activation functions in the hidden layers
- number of epochs
- number of features to include in the model
- etc.

Let's start by importing some basic libraries and the data:

In [ ]:
## import libraries
import numpy as np
import tensorflow as tf
import pandas as pd
import sklearn.datasets
import matplotlib.pyplot as plt


## Breast cancer data

Now, it's up to you to continue: write here your code!! (plus text chunks for explanations)

In [ ]:
from sklearn.datasets import load_breast_cancer
bcancer = load_breast_cancer()
y = bcancer.target
X = bcancer.data
y.shape

In [ ]:
from collections import Counter
print(Counter(y))

In [ ]:
print(bcancer.DESCR)

### Explore the data

In [ ]:
bcancer.data = pd.DataFrame(bcancer.data, columns=bcancer.feature_names) #converting numpy array -> pandas DataFrame
bcancer.target = pd.Series(bcancer.target)

In [ ]:
features = bcancer.data.iloc[:,:]
target = bcancer.target
features

In [ ]:
#we want to have the same proportion of classes in both train and validation sets
from sklearn.model_selection import StratifiedShuffleSplit

#building a StratifiedShuffleSplit object (sss among friends) with 20% data
#assigned to validation set (here called "test")
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

#the .split() method returns (an iterable over) two lists which can be
#used to index the samples that go into train and validation sets
for train_index, val_index in sss.split(features, target):
    X_train = features.iloc[train_index, :]
    X_val   = features.iloc[val_index, :]
    y_train   = target[train_index]
    y_val     = target[val_index]
    
#let's print some shapes to get an idea of the resulting data structure
print("Training features size: ", X_train.shape)
print("Test features size: ", X_val.shape)
print("Training targets size: ", y_train.shape)
print("Test targets size: ", y_val.shape)

print("Type of the training features object: ", type(X_train))
print("Type of the training targets object: ", type(y_train))

In [ ]:
## # Configuration options
input_shape = (X_train.shape[1],) ## tuple that specifies the number of features 
hidden_nodes = 16
hidden_activation = 'relu'
output_activation = 'sigmoid'
loss_function = 'binary_crossentropy'
optimizer_used = 'sgd' ##stochastic gradient descent
num_epochs = 100

print(input_shape)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense ## a "dense" layer is a layer were all the data coming in are connected
#to all nodes.

# binary classification shallow neural network model in Keras
model = Sequential()
model.add(Dense(units=hidden_nodes, input_shape=input_shape, activation=hidden_activation))
model.add(Dense(1, activation=output_activation))

#the model is declared, but we still need to compile it to actually
#build all the data structures
model.compile(optimizer=optimizer_used, loss=loss_function)

In [ ]:
print(model.summary())

In [ ]:
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_val, y_val), verbose=0)

In [ ]:
def plot_loss_history(h, title):
    plt.plot(h.history['loss'], label = "Train loss")
    plt.plot(h.history['val_loss'], label = "Validation loss")
    plt.xlabel('Epochs')
    plt.title(title)
    plt.legend()
    plt.show() 

plot_loss_history(history, 'Logistic ({} epochs)'.format(num_epochs))

In [ ]:
from sklearn.metrics import confusion_matrix

predictions = model.predict(X_val)
predicted_labels = np.where(predictions > 0.5, "benign", "malignant")
target_labels = y_val.to_numpy().reshape((len(y_val),1))
target_labels = np.where(target_labels > 0.5, "benign", "malignant")

con_mat_df = confusion_matrix(target_labels, predicted_labels, labels=["malignant","benign"])
print(con_mat_df)

### Data normalization

In [ ]:
features.head()

In [ ]:
#getting an idea about features averages, sd
avg = X_train.mean()
std = X_train.std()
print('Feature means')
print(avg)
print('\nFeature standard deviations')
print(std)

In [ ]:
#IMPORTANT: normalizing features using the same weights for both
#train and validation test (which are computed ON THE TRAIN SET)
X_train = (X_train - avg)/std
X_val = (X_val - avg)/std

In [ ]:
X_train

In [ ]:
## # Configuration options
input_shape = (X_train.shape[1],) ## tuple that specifies the number of features 
hidden_nodes = 16
hidden_activation = 'relu'
output_activation = 'sigmoid'
loss_function = 'binary_crossentropy'
optimizer_used = 'sgd' ##stochastic gradient descent
num_epochs = 100

In [ ]:
#we are building a "sequential" model, meaning that the data will 
#flow like INPUT -> ELABORATION -> OUTPUT.
from keras.models import Sequential

#a "dense" layer is a layer were all the data coming in are connected
#to all nodes.
from keras.layers import Dense

# binary classification shallow neural network model in Keras
model = Sequential()
model.add(Dense(units=hidden_nodes, input_shape=input_shape, activation=hidden_activation))
model.add(Dense(1, activation=output_activation))

#the model is declared, but we still need to compile it to actually
#build all the data structures
model.compile(optimizer=optimizer_used, loss=loss_function)
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_val, y_val), verbose=0)

In [ ]:
plot_loss_history(history, 'Logistic ({} epochs)'.format(num_epochs))

In [ ]:
predictions = model.predict(X_val)
predicted_labels = np.where(predictions > 0.5, "cancer", "no-cancer")
target_labels = y_val.to_numpy().reshape((len(y_val),1))
target_labels = np.where(target_labels > 0.5, "cancer", "no-cancer")

con_mat_df = confusion_matrix(target_labels, predicted_labels, labels=["no-cancer","cancer"])
print(con_mat_df)

In [ ]:
history2 = model.fit(X_train, y_train, epochs=100, 
                     validation_data=(X_val, y_val), verbose=0)

In [ ]:
#putting together the whole history
history.history['loss'] += history2.history['loss']
history.history['val_loss'] += history2.history['val_loss']

#and plotting again
plot_loss_history(history, 'Logistic (500 epochs)')

In [ ]:
predictions = model.predict(X_val)
predicted_labels = np.where(predictions > 0.5, "cancer", "no-cancer")
target_labels = y_val.to_numpy().reshape((len(y_val),1))
target_labels = np.where(target_labels > 0.5, "cancer", "no-cancer")

con_mat_df = confusion_matrix(target_labels, predicted_labels, labels=["no-cancer","cancer"])
print(con_mat_df)

In [ ]:
model = Sequential()
model.add(Dense(units=hidden_nodes, input_shape=input_shape, activation=hidden_activation))
model.add(Dense(units=hidden_nodes, input_shape=input_shape, activation=hidden_activation))
model.add(Dense(1, activation=output_activation))

#the model is declared, but we still need to compile it to actually
#build all the data structures
model.compile(optimizer=optimizer_used, loss=loss_function)
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=num_epochs, validation_data=(X_val, y_val), verbose=0)

In [ ]:
plot_loss_history(history, 'Breast cancer data (100 epochs)')

In [ ]:
predictions = model.predict(X_val)
predicted_labels = np.where(predictions > 0.5, "cancer", "no-cancer")
target_labels = y_val.to_numpy().reshape((len(y_val),1))
target_labels = np.where(target_labels > 0.5, "cancer", "no-cancer")

con_mat_df = confusion_matrix(target_labels, predicted_labels, labels=["no-cancer","cancer"])
print(con_mat_df)

In [ ]:
import seaborn as sn

figure = plt.figure(figsize=(8, 8))
sn.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()